In [6]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# Load the uploaded dataset
data = pd.read_csv('Iris.csv')

# Display the first few rows of the dataset to understand its structure
data.head()


,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Class Label
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


### Preprocessing

In [7]:

X = data.iloc[:, :-1].values  # Features (sepal_length, sepal_width, petal_length, petal_width)
y = data.iloc[:, -1].values   # Target (species)

In [8]:
# Encode target labels to one-hot format
encoder = OneHotEncoder()
y = encoder.fit_transform(y.reshape(-1, 1)).toarray()

In [9]:
# Standardize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [10]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


###  Define the Neural Network Architecture

In [11]:

input_size = X.shape[1]  # Number of input features
hidden_size = 5          # Number of neurons in the hidden layer
output_size = y.shape[1] # Number of output classes (species)

### Initialize weights and biases

In [12]:

np.random.seed(42)
weights_input_hidden = np.random.randn(input_size, hidden_size)
bias_hidden = np.zeros((1, hidden_size))
weights_hidden_output = np.random.randn(hidden_size, output_size)
bias_output = np.zeros((1, output_size))

### Activation functions

In [13]:

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

def softmax(x):
    exp_values = np.exp(x - np.max(x, axis=1, keepdims=True))
    return exp_values / np.sum(exp_values, axis=1, keepdims=True)

### Loss function

In [14]:
def cross_entropy_loss(y_true, y_pred):
    return -np.mean(np.sum(y_true * np.log(y_pred + 1e-9), axis=1))

### Forward pass

In [15]:

def forward_pass(X):
    hidden_input = np.dot(X, weights_input_hidden) + bias_hidden
    hidden_output = sigmoid(hidden_input)

    output_input = np.dot(hidden_output, weights_hidden_output) + bias_output
    output_output = softmax(output_input)

    return hidden_output, output_output

### Backpropagation

In [16]:

def backward_pass(X, y, hidden_output, output_output, learning_rate):
    global weights_input_hidden, bias_hidden, weights_hidden_output, bias_output

    # Output layer error
    output_error = output_output - y
    output_gradient = np.dot(hidden_output.T, output_error)

    # Hidden layer error
    hidden_error = np.dot(output_error, weights_hidden_output.T) * sigmoid_derivative(hidden_output)
    hidden_gradient = np.dot(X.T, hidden_error)

    # Update weights and biases
    weights_hidden_output -= learning_rate * output_gradient
    bias_output -= learning_rate * np.sum(output_error, axis=0, keepdims=True)
    weights_input_hidden -= learning_rate * hidden_gradient
    bias_hidden -= learning_rate * np.sum(hidden_error, axis=0, keepdims=True)

### Training the Neural Network

In [17]:

epochs = 1000
learning_rate = 0.01

for epoch in range(epochs):
    # Forward pass
    hidden_output, output_output = forward_pass(X_train)

    # Compute loss
    loss = cross_entropy_loss(y_train, output_output)

    # Backward pass
    backward_pass(X_train, y_train, hidden_output, output_output, learning_rate)

    if epoch % 100 == 0:
        print(f"Epoch {epoch}, Loss: {loss:.4f}")

Epoch 0, Loss: 2.1255
Epoch 100, Loss: 0.0615
Epoch 200, Loss: 0.0245
Epoch 300, Loss: 0.0145
Epoch 400, Loss: 0.0101
Epoch 500, Loss: 0.0077
Epoch 600, Loss: 0.0062
Epoch 700, Loss: 0.0051
Epoch 800, Loss: 0.0044
Epoch 900, Loss: 0.0038


### Testing the Model

In [18]:

_, y_pred = forward_pass(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

accuracy = np.mean(y_pred_classes == y_true_classes)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Test Accuracy: 100.00%
